In [1]:
import numpy as np

In [80]:
class Conv2D:
    
    def __init__(self, ksize, filters, input_shape, activation, stride=1, padding=0):
        self.ksize = ksize
        self.filters = filters
        self.stride = stride
        self.padding = padding
        self.input_shape = input_shape
        self.channels = input_shape[-1]
        self.activation = activation
        self.kernels = []
        for i in range(self.filters):
            k = np.random.randn(ksize, ksize, self.channels)
            self.kernels.append(k)
        self.bias = np.random.randn(1,self.filters)
        
    @staticmethod
    def _rotate(inp):
        assert len(inp.shape)==4, f"No. of dim in inp not equal to 4, got {inp.shape}"
        return np.flip(inp, axis=(1,2))

    @staticmethod
    def _convolution_op_helper(inp, kernel, stride=1):
        # inp shape -> 4 dim
        assert len(inp.shape)==4, f"No. of dim in inp not equal to 4, got {inp.shape}"
        # kernel shouldhave 4 dim
        assert len(kernel.shape)==4, f"No. of dim in kernel not equal to 4, got {kernel.shape}"

        # no. of chanels in kernel and that in inp it should be same
        assert inp.shape[-1] == kernel.shape[-1], f"Mismatch in no. of channels in inp and kernel, got inp {inp.shape[-1]}, kernel {kernel.shape[-1]}"
        # non-square kernels are not allowed
        assert kernel.shape[1] == kernel.shape[2], f"dim 0 of kernel doesn't match dim 1, got {kernel.shape}"
        # inp shape square
        assert inp.shape[1]>=kernel.shape[1] and inp.shape[2]>=kernel.shape[2], f"Inp map dim(1,2) < kernel dim(1,2), got inp map dim 1, 2 {inp.shape[1:-1]}, kernel dim 1,2 {kernel.shape[1:-1]}"

        # flip the kernel
        kernel = Conv2D._rotate(kernel)

        oup = []
        start_rloc = 0
        end_rloc = kernel.shape[1]
        while end_rloc <= inp.shape[1]:
            output = []
            start_cloc = 0
            end_cloc = kernel.shape[2]
            while end_cloc <= inp.shape[2]:
                conv = (inp[:,start_rloc:end_rloc, start_cloc:end_cloc]*kernel).sum(axis=(1,2,3))
                output.append(conv)

                start_cloc += stride
                end_cloc += stride
            oup.append(output)
            start_rloc += stride
            end_rloc += stride
        return np.moveaxis(oup, -1, 0)
    
    def _convolution_op(self, inp):
        output = []
        for kernel in self.kernels:
            o = Conv2D._convolution_op_helper(inp, np.expand_dims(kernel, axis=0), self.stride)
            output.append(o)
        return np.stack(output, axis=-1)
            
    @staticmethod
    def _pad(inp, pad_width):   
        assert len(inp.shape)==4, f"No. of dim in inp not equal to 4, got {inp.shape}"
        return np.pad(inp, ((0,0), (pad_width,pad_width), (pad_width,pad_width), (0,0)))

    @staticmethod
    def _inside_pad(inp, pad_width):
        assert len(inp.shape)==4, f"No. of dim in inp not equal to 4, got {inp.shape}"
        ix = np.repeat(np.arange(1, inp.shape[1]), pad_width)
        inp = np.insert(inp, ix, 0, axis=1)
        return np.insert(inp, ix, 0, axis=2)
        

    def eval(self, X):
        o_ = self._convolution_op(X) + self.bias
        return self.activation(o_)

    def grad_input(self, X):
        g1 = self.activation.grad_input( self.dot(X) )
        g2 = self.dot.grad_input(X)
        return np.einsum('mij,mjk->mik', g1, g2)

    def grad_parameters(self, X):
        da_dI = self.activation.grad_input(self.dot(X))
        dI_dw = self.dot.grad_w(X)
        da_dw = np.einsum('mij,mjkl->mikl', da_dI, dI_dw)

        dI_db = self.dot.grad_b(X)
        # print(da_dI.shape, dI_dw.shape, dI_db.shape)
        da_db = np.einsum('mij,mjk->mik',  da_dI, dI_db)
        return da_dw, da_db

    def backprop_grad(self, grad_loss, grad):
        dL_dwi = np.einsum('mij,mjkl->mikl', grad_loss, grad['w']).sum(axis=0)
        dL_dbi = np.einsum('mij,mjk->mik', grad_loss, grad['b']).sum(axis=0)
        grad_loss = np.einsum('mij,mjk->mik', grad_loss, grad['input'])
        return dL_dwi, dL_dbi, grad_loss
        
    def update(self, grad, optimizer):
        """ grad: (dL_dwi, dL_dbi)"""
        self.dot.W = optimizer.minimize(self.dot.W, grad[0])
        self.dot.b = optimizer.minimize(self.dot.b, grad[1])
        
    def get_parameter_shape(self):
        return self.dot.get_parameter_shape()
    
    def get_output_shape(self):
        m, n, k, p, s = self.input_shape[1], input_shape[2], self.ksize, self.padding, self.stride
        return (m-k+(2*p)//s)+1, (n-k+(2*p)//s)+1
    
    def get_total_parameters(self):
        return np.prod((len(self.kernels), *self.kernels[0].shape)) + np.prod(self.bias.shape)

In [78]:
class Sigmoid:

    def __call__(self, X):
        return self.eval(X)

    def eval(self, X):
        return 1/((np.e**-X) + 1)

    def grad_input(self, X):
        I = np.identity(X.shape[1])
        b = self.eval(X)*(1-self.eval(X)) # same shape as X
        return np.einsum('ij,mi->mij', I, b)


In [82]:
inp = np.ones((10, 10,10, 3))
l1 = Conv2D(ksize=5, filters=2, input_shape=inp.shape, activation=Sigmoid(), stride=1, padding=0)
o = l1.eval(inp)


In [83]:
o.shape

(10, 6, 6, 2)

In [84]:
l2 = Conv2D(ksize=5, filters=2, input_shape=o.shape, activation=Sigmoid(), stride=1, padding=0)
l2.eval(o).shape

(10, 2, 2, 2)